# Clase 3 — Desarrollo de aplicaciones con Visión por Computador

---

## Objetivos de aprendizaje

Al finalizar la sesión, los estudiantes serán capaces de:

1. Diseñar una API REST segura y eficiente para servir modelos de visión por computador.
2. Implementar endpoints para clasificación y detección que sigan buenas prácticas (validación, manejo de errores, tipado).
3. Exportar un modelo PyTorch a TorchScript u ONNX y explicar las ventajas de cada formato para producción.
4. Contenerizar la aplicación con Docker y describir consideraciones para despliegue en producción.

---



## Requisitos previos y recursos

* Python 3.9+
* PyTorch + torchvision
* FastAPI, Uvicorn

Ejemplo mínimo de 

`requirements.txt`:

```
fastapi>=0.95
uvicorn[standard]>=0.20
torch>=1.12
torchvision>=0.13
pydantic>=1.10
numpy>=1.23
python-multipart>=0.0.5
pytest>=7.0
requests>=2.28
```

---



## Parte teórica

### 1) ¿Por qué separar modelo y aplicación?

* **Separación de responsabilidades**: modelo (inferencia) vs API (validación, seguridad, logging).
* **Escalabilidad**: independizar el servicio permite escalar el modelo por separado (más réplicas o GPU nodes).
* **Observabilidad**: el servicio web permite instrumentación (métricas, logs, traces).

### 2) Opciones de serving

* **FastAPI**: rápido de desarrollar, documentación automática (OpenAPI), ideal para PoC y microservicios. Permite endpoints sin complicaciones.
* **TorchServe**: diseñado para modelos PyTorch en producción; soporta batching, métricas y gestion de endpoints con modelos múltiples.
* **TensorFlow Serving / TF-Serving**: equivalente para modelos TensorFlow.
* **ONNX Runtime**: para ejecutar modelos exportados a ONNX con alto rendimiento en CPU y aceleradores.

**Regla**: para prototipos y clases use FastAPI; para producción con necesidades de escalado/batching considere TorchServe/ONNX.

### 3) Requisitos de diseño de API

* Endpoints claros: `/health`, `/predict/classify`, `/predict/detect`.
* Validación de entrada con Pydantic (tamaños máximos, tipos, formatos MIME permitidos).
* Soporte para subida de archivos (`multipart/form-data`) y payloads base64.
* Respuestas JSON estandarizadas (clave `status`, `predictions`, `error`, `meta`).

### 4) Seguridad y privacidad mínima

* Limitar tamaño de subida (`max_content_length`), evitar DoS por archivos grandes.
* Autenticación: JWT / API keys para acceso a la API en producción.
* Si se procesan imágenes con datos personales: anonimización, políticas de retención y cumplimiento (GDPR/HIPAA según correspondencia).

---



## Laboratorio práctico — Integración de una CNN en un servicio web

### Objetivo práctico

Construir un servicio web (FastAPI) que:

1. Cargue un modelo PyTorch (preentrenado o fine-tuned).
2. Exponga un endpoint `/predict/classify` que acepte una imagen y devuelva la clase y probabilidades.
3. (Opcional) Exponga `/predict/detect` que devuelva bounding boxes y scores usando un detector preentrenado.

### Estructura de archivos sugerida

```
cv-app/
├── app/
│   ├── main.py            # FastAPI app
│   ├── model.py           # carga, preproc y postproc del modelo
│   └── schemas.py         # Pydantic models para requests/responses
├── requirements.txt
```

> Recomendación: mantenga la lógica de preprocesado y postprocesado en `model.py` para facilitar tests y posible exportación.

### Código: `app/model.py` (ejemplo para clasificación con ResNet18)



In [ ]:
# ------------------------------------------------------------
# app/model.py — Módulo del modelo de clasificación de imágenes
# ------------------------------------------------------------

# Permite compatibilidad futura con anotaciones de tipo (Python 3.7+)
from __future__ import annotations

# Importaciones estándar
import io          # Para manipular datos binarios (como imágenes en bytes)
import logging     # Para registrar mensajes informativos, advertencias y errores
from typing import List, Tuple  # Tipos usados para mayor claridad en el código
import json        # Librería estándar para manejar datos en formato JSON

# Librerías de procesamiento científico y deep learning
import numpy as np                  # Para operaciones numéricas y manejo de arrays
import torch                        # Framework de aprendizaje profundo PyTorch
import torchvision.transforms as T  # Módulo de transformaciones de imágenes
from PIL import Image               # PIL (Pillow): para manipulación de imágenes
from torchvision import models       # Modelos preentrenados y arquitecturas CNN

# Crear un logger específico para este módulo
logger = logging.getLogger("cv_app.model")

# ------------------------------------------------------------
# Clase principal: ImageClassifier
# ------------------------------------------------------------
class ImageClassifier:
    """
    Clase que encapsula la lógica de un modelo de clasificación de imágenes.
    Utiliza MobileNetV3 preentrenado en ImageNet para clasificar imágenes.
    """

    def __init__(self, device: str = "cpu") -> None:
        """
        Constructor del clasificador.
        Carga el modelo, las etiquetas y define las transformaciones.
        """
        self.device = device  # CPU o GPU según disponibilidad

        # Cargar el modelo preentrenado
        self.model = self._load_model()

        # Mover el modelo al dispositivo correspondiente (CPU o GPU)
        self.model.to(self.device)

        # Poner el modelo en modo evaluación (no entrenamiento)
        self.model.eval()

        # Cargar las etiquetas de las clases de ImageNet
        self.labels = self._load_imagenet_labels()

        # Definir las transformaciones de preprocesamiento de imágenes
        # Estas son las normalizaciones estándar usadas por ImageNet
        self.transform = T.Compose([
            T.Resize(224),  # Redimensiona la imagen al tamaño 224x224
            T.ToTensor(),   # Convierte la imagen a tensor de PyTorch (valores [0,1])
            T.Normalize(    # Normaliza los valores RGB con la media y desviación típicas
                mean=[0.485, 0.456, 0.406],   # Medias por canal RGB
                std=[0.229, 0.224, 0.225]     # Desviaciones estándar por canal
            ),
        ])

    # --------------------------------------------------------
    # Función interna para cargar el modelo
    # --------------------------------------------------------
    def _load_model(self) -> torch.nn.Module:
        """
        Descarga y carga el modelo preentrenado MobileNetV3-Small.
        Es un modelo liviano y eficiente para clasificación de imágenes.
        """
        logger.info("Descargando modelo MobileNet (ligero)...")

        # Se descarga y carga el modelo con pesos preentrenados en ImageNet
        model = models.mobilenet_v3_small(
            weights=models.MobileNet_V3_Small_Weights.IMAGENET1K_V1
        )

        logger.info("Modelo MobileNet descargado y cargado correctamente")

        # Si se quisiera adaptar el modelo a otro número de clases,
        # aquí se modificaría la capa de salida (classifier).
        return model

    # --------------------------------------------------------
    # Cargar etiquetas de ImageNet
    # --------------------------------------------------------
    def _load_imagenet_labels(self) -> List[str]:
        """
        Carga las etiquetas (nombres de clases) del dataset ImageNet.
        Estas etiquetas están incluidas dentro de los pesos del modelo.
        """
        try:
            # Se obtienen los pesos del modelo MobileNetV3
            from torchvision.models import get_model_weights

            # Cargar las etiquetas desde los metadatos de los pesos
            weights = get_model_weights('mobilenet_v3_small')
            labels = weights.IMAGENET1K_V1.meta['categories']

            logger.info(f"Cargadas {len(labels)} etiquetas desde weights preentrenados")
            return labels

        except Exception as e:
            # Si ocurre un error (por ejemplo, sin conexión a internet)
            logger.warning(f"No se pudieron cargar las etiquetas desde weights: {e}")

            # Se devuelven etiquetas genéricas de respaldo (objeto_0, objeto_1, …)
            return [f"objeto_{i}" for i in range(1000)]

    # --------------------------------------------------------
    # Preprocesamiento de imágenes
    # --------------------------------------------------------
    def preprocess(self, image_bytes: bytes) -> torch.Tensor:
        """
        Convierte los bytes de una imagen en un tensor listo para el modelo.
        - Decodifica los bytes a imagen RGB
        - Aplica transformaciones (resize, normalización, tensor)
        - Añade una dimensión batch (1 x 3 x 224 x 224)
        """
        # Abre la imagen desde bytes usando PIL y la convierte a RGB
        image = Image.open(io.BytesIO(image_bytes)).convert("RGB")

        # Aplica las transformaciones definidas (resize, normalize, etc.)
        x = self.transform(image).unsqueeze(0)  # unsqueeze añade la dimensión batch

        # Devuelve el tensor en el dispositivo correcto (CPU o GPU)
        return x.to(self.device)

    # --------------------------------------------------------
    # Predicción
    # --------------------------------------------------------
    @torch.inference_mode()  # Desactiva gradientes para optimizar inferencia
    def predict(self, image_bytes: bytes, topk: int = 5) -> List[Tuple[str, float]]:
        """
        Realiza la predicción de clase sobre una imagen.
        Devuelve las top-K clases más probables con sus probabilidades.
        """
        # Preprocesar la imagen (de bytes a tensor normalizado)
        x = self.preprocess(image_bytes)

        # Pasar la imagen por el modelo (forward pass)
        logits = self.model(x)

        # Aplicar softmax para convertir logits en probabilidades
        probs = torch.nn.functional.softmax(logits, dim=1)

        # Seleccionar las K clases con mayor probabilidad
        top_probs, top_idxs = probs.topk(topk, dim=1)

        # Convertir resultados a listas de Python para facilidad de uso
        top_probs = top_probs.cpu().numpy().flatten().tolist()
        top_idxs = top_idxs.cpu().numpy().flatten().tolist()

        # Mapear los índices a las etiquetas reales de ImageNet
        labels = [self.labels[idx] for idx in top_idxs]

        # Devolver pares (etiqueta, probabilidad)
        return list(zip(labels, top_probs))


### Código: `app/schemas.py` (Pydantic)



In [ ]:
# ------------------------------------------------------------
# app/schemas.py — Definición de esquemas de datos con Pydantic
# ------------------------------------------------------------
# En este módulo se definen las estructuras de datos (modelos)
# que FastAPI usa para validar, documentar y serializar la entrada/salida
# de la API. Los esquemas aseguran que los datos sigan un formato consistente.

# ------------------------------------------------------------
# Importaciones necesarias
# ------------------------------------------------------------
from pydantic import BaseModel, Field  # Base para crear modelos de validación
from typing import List, Tuple         # Tipos de datos para listas y tuplas

# ------------------------------------------------------------
# Modelo Prediction
# ------------------------------------------------------------
class Prediction(BaseModel):
    """
    Representa una predicción individual del modelo de clasificación.
    Cada predicción tiene:
    - label: nombre de la clase predicha (por ejemplo, 'perro', 'auto')
    - score: probabilidad asociada a esa predicción (float entre 0 y 1)
    """
    label: str    # Nombre de la clase predicha
    score: float  # Probabilidad asociada a la predicción

# ------------------------------------------------------------
# Modelo ClassifyResponse
# ------------------------------------------------------------
class ClassifyResponse(BaseModel):
    """
    Define la estructura de la respuesta JSON del endpoint de clasificación.
    Incluye:
    - status: texto que indica el estado de la petición (por defecto "ok")
    - predictions: lista de objetos Prediction (las predicciones del modelo)
    """
    status: str = Field("ok")              # Campo con valor por defecto "ok"
    predictions: List[Prediction]          # Lista de predicciones generadas


### Código: `app/main.py` (FastAPI app)



In [ ]:
# ------------------------------------------------------------
# app/main.py — API REST para Clasificación de Imágenes
# ------------------------------------------------------------
# Este módulo implementa una aplicación web usando FastAPI.
# Permite cargar imágenes y clasificarlas mediante un modelo
# de Deep Learning (MobileNet preentrenado en ImageNet).
# Se siguen principios de:
# - PEP 8 (estilo de código en Python)
# - RESTful API Design
# - Clean Code y separación de responsabilidades
# ------------------------------------------------------------

# ------------------------------------------------------------
# Importaciones necesarias
# ------------------------------------------------------------
from fastapi import FastAPI, File, UploadFile, HTTPException  # Framework web moderno y rápido
from fastapi.responses import JSONResponse                    # Permite devolver respuestas JSON personalizadas
import logging                                                 # Módulo estándar de logging (registro de eventos)

# ------------------------------------------------------------
# Importaciones de módulos locales (de la carpeta app/)
# ------------------------------------------------------------
from model.model import ImageClassifier                       # Clase para manejar el modelo de IA
from schemas.schema import ClassifyResponse, Prediction        # Modelos Pydantic para la estructura de respuesta

# ------------------------------------------------------------
# Configuración inicial del logger
# ------------------------------------------------------------
logger = logging.getLogger("cv_app")  # Crea un logger llamado "cv_app" para registrar mensajes en consola o archivo

# ------------------------------------------------------------
# Inicialización de la aplicación FastAPI
# ------------------------------------------------------------
app = FastAPI(
    title="CV Model API",   # Título visible en la documentación automática (/docs)
    version="0.1"           # Versión de la API
)

# ------------------------------------------------------------
# Carga del modelo de manera diferida (lazy loading)
# ------------------------------------------------------------
# La idea es no cargar el modelo de IA al iniciar la API,
# sino solo cuando se necesita por primera vez (optimización de recursos).
classifier: ImageClassifier | None = None  # Variable global para almacenar el modelo

def get_classifier():
    """
    Carga el modelo de clasificación de imágenes solo una vez.
    Si ya está cargado, lo reutiliza.
    Esta técnica evita que el modelo se cargue múltiples veces,
    lo que mejora la eficiencia y reduce el uso de memoria.
    """
    global classifier  # Se usa global para acceder/modificar la variable definida arriba

    if classifier is None:  # Si el modelo no está cargado aún
        logger.info("Cargando modelo por primera vez...")  # Mensaje informativo
        classifier = ImageClassifier(device="cpu")         # Inicializa el modelo en CPU
        logger.info("Modelo cargado exitosamente")         # Confirma carga exitosa

    return classifier  # Devuelve la instancia del modelo (ya cargada)


# ------------------------------------------------------------
# Endpoint raíz: información general de la API
# ------------------------------------------------------------
@app.get("/")
def root():
    """Endpoint raíz con descripción general de la API."""
    return {
        "message": "API de Clasificación de Imágenes con IA",  # Mensaje principal
        "version": "1.0.0",                                    # Versión informativa
        "description": "Clasifica imágenes usando MobileNet pre-entrenado en ImageNet",
        # Información de los endpoints disponibles
        "endpoints": {
            "health": "/health - Estado del servidor",
            "classify": "/predict/classify - Clasificar una imagen",
            "test_labels": "/test-labels - Ver etiquetas del modelo",
            "docs": "/docs - Documentación interactiva generada por FastAPI"
        },
        # Ejemplo de uso para usuarios nuevos
        "usage": {
            "method": "POST",
            "endpoint": "/predict/classify",
            "content_type": "multipart/form-data",
            "parameter": "file (imagen)"
        },
        # Ejemplo práctico de cómo consumir el servicio
        "example": "Sube una imagen a /predict/classify para obtener las 3 predicciones más probables"
    }


# ------------------------------------------------------------
# Endpoint de salud: verificación del estado del servidor
# ------------------------------------------------------------
@app.get("/health")
def health():
    """
    Devuelve el estado básico del servidor.
    Permite saber si la API está corriendo correctamente.
    """
    return {"status": "ok"}  # Respuesta simple tipo JSON


# ------------------------------------------------------------
# Endpoint de prueba de etiquetas del modelo
# ------------------------------------------------------------
@app.get("/test-labels")
def test_labels():
    """
    Devuelve una muestra de etiquetas del modelo preentrenado.
    Sirve para verificar que el modelo se cargó correctamente
    y que las clases están disponibles.
    """
    classifier = get_classifier()                # Asegura que el modelo esté cargado
    sample_labels = classifier.labels[:10]       # Obtiene las primeras 10 etiquetas
    return {
        "labels_sample": sample_labels,          # Ejemplo de etiquetas
        "total_labels": len(classifier.labels)   # Número total de etiquetas del modelo
    }


# ------------------------------------------------------------
# Endpoint principal: clasificación de imágenes
# ------------------------------------------------------------
@app.post("/predict/classify", response_model=ClassifyResponse)
async def predict_classify(file: UploadFile = File(...)):
    """
    Endpoint para clasificar una imagen.
    Recibe una imagen en formato multipart/form-data,
    la procesa con el modelo y devuelve las predicciones más probables.

    Parámetros:
    -----------
    file : UploadFile
        Imagen enviada por el usuario.

    Retorna:
    --------
    ClassifyResponse
        Estructura JSON con el estado y las predicciones (label y score).
    """

    # --------------------------------------------------------
    # Validación del tipo de archivo
    # --------------------------------------------------------
    if not file.content_type.startswith("image/"):
        # Si el archivo no es una imagen, se lanza un error HTTP 400 (Bad Request)
        raise HTTPException(status_code=400, detail="Archivo no es una imagen")

    # --------------------------------------------------------
    # Lectura de bytes del archivo
    # --------------------------------------------------------
    image_bytes = await file.read()  # Lee el contenido binario del archivo (asíncrono)

    # --------------------------------------------------------
    # Validación de contenido vacío
    # --------------------------------------------------------
    if len(image_bytes) == 0:
        raise HTTPException(status_code=400, detail="Archivo vacío")

    # --------------------------------------------------------
    # Carga del modelo solo cuando se necesita
    # --------------------------------------------------------
    classifier = get_classifier()

    # --------------------------------------------------------
    # Generación de predicciones
    # --------------------------------------------------------
    preds = classifier.predict(image_bytes, topk=3)  # topk=3: se devuelven las 3 clases más probables

    # --------------------------------------------------------
    # Construcción de la respuesta usando Pydantic
    # --------------------------------------------------------
    response = ClassifyResponse(
        predictions=[Prediction(label=p[0], score=p[1]) for p in preds]
    )

    # --------------------------------------------------------
    # Devolución de la respuesta estructurada
    # --------------------------------------------------------
    return response


# Ahora...

## Crear la carpeta /img y guardar las imagenes para probar el modelo

## Ejecutar la aplicación
```
uvicorn main:app --reload --host 127.0.0.1 --port 8000
```
### **2. En el navegador web:**
- **Documentación**: `http://127.0.0.1:8000/docs`
- **API principal**: `http://127.0.0.1:8000`
- **Health check**: `http://127.0.0.1:8000/health`

### **3. Para probar la clasificación de imágenes:**

1. **Abre tu navegador**
2. **Ve a**: `http://127.0.0.1:8000/docs`
3. **Verás la interfaz de Swagger** con los endpoints
4. **Haz clic en** `/predict/classify`
5. **Haz clic en** "Try it out"
6. **Sube una imagen** y haz clic en "Execute"

### **4. La consola mostrará:**
- Cuando alguien haga una petición
- Los logs de carga del modelo (primera vez)
- Los logs de procesamiento de imágenes



## Lecturas y recursos

* FastAPI docs (Auto-generated OpenAPI).
* PyTorch/TorchVision model zoo.
* TorchServe docs y ejemplos para serving en producción.
* ONNX Runtime docs.

